In [1]:
########################################
# CS/CNS/EE 155 2017
# Problem Set 5
#
# Author:       Andrew Kang
# Description:  Set 5 solutions
########################################

import random
import preprocessing as pp

#creates a dictionary of all the words in the syllable dictionary, gives them
#an index
words = pp.word_id()
#also creates an inverse dictionary for reverse lookup
word_list = pp.list_of_words()
#word to number of syllables dictionary
word_sylls = pp.word_syllables()

class HiddenMarkovModel:
    '''
    Class implementation of Hidden Markov Models.
    '''

    def __init__(self, A, O):
        '''
        Initializes an HMM. Assumes the following:
            - States and observations are integers starting from 0.
            - There is a start state (see notes on A_start below). There
              is no integer associated with the start state, only
              probabilities in the vector A_start.
            - There is no end state.

        Arguments:
            A:          Transition matrix with dimensions L x L.
                        The (i, j)^th element is the probability of
                        transitioning from state i to state j. Note that
                        this does not include the starting probabilities.

            O:          Observation matrix with dimensions L x D.
                        The (i, j)^th element is the probability of
                        emitting observation j given state i.

        Parameters:
            L:          Number of states.

            D:          Number of observations.

            A:          The transition matrix.

            O:          The observation matrix.

            A_start:    Starting transition probabilities. The i^th element
                        is the probability of transitioning from the start
                        state to state i. For simplicity, we assume that
                        this distribution is uniform.
        '''

        self.L = len(A)
        self.D = len(O[0])
        self.A = A
        self.O = O
        self.A_start = [1. / self.L for _ in range(self.L)]


    def viterbi(self, x):
        '''
        Uses the Viterbi algorithm to find the max probability state
        sequence corresponding to a given input sequence.

        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.

        Returns:
            max_seq:    Output sequence corresponding to x with the highest
                        probability.
        '''

        M = len(x)      # Length of sequence.

        # The (i, j)^th elements of probs and seqs are the max probability
        # of the prefix of length i ending in state j and the prefix
        # that gives this probability, respectively.
        #
        # For instance, probs[1][0] is the probability of the prefix of
        # length 1 ending in state 0.
        probs = [[0. for _ in range(self.L)] for _ in range(M + 1)]
        seqs = [['' for _ in range(self.L)] for _ in range(M + 1)]

        # Calculate initial prefixes and probabilities.
        for curr in range(self.L):
            probs[1][curr] = self.A_start[curr] * self.O[curr][x[0]]
            seqs[1][curr] = str(curr)

        # Calculate best prefixes and probabilities throughout sequence.
        for t in range(2, M + 1):
            # Iterate over all possible current states.
            for curr in range(self.L):
                max_prob = float("-inf")
                max_prefix = ''

                # Iterate over all possible previous states to find one
                # that would maximize the probability of the current state.
                for prev in range(self.L):
                    curr_prob = probs[t - 1][prev] \
                                * self.A[prev][curr] \
                                * self.O[curr][x[t - 1]]

                    # Continually update max probability and prefix.
                    if curr_prob >= max_prob:
                        max_prob = curr_prob
                        max_prefix = seqs[t - 1][prev]

                # Store the max probability and prefix.
                probs[t][curr] = max_prob
                seqs[t][curr] = max_prefix + str(curr)

        # Find the index of the max probability of a sequence ending in x^M
        # and the corresponding output sequence.
        max_i = max(enumerate(probs[-1]), key=lambda x: x[1])[0]
        max_seq = seqs[-1][max_i]

        return max_seq


    def forward(self, x, normalize=False):
        '''
        Uses the forward algorithm to calculate the alpha probability
        vectors corresponding to a given input sequence.

        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.

            normalize:  Whether to normalize each set of alpha_j(i) vectors
                        at each i. This is useful to avoid underflow in
                        unsupervised learning.

        Returns:
            alphas:     Vector of alphas.

                        The (i, j)^th element of alphas is alpha_j(i),
                        i.e. the probability of observing prefix x^1:i
                        and state y^i = j.

                        e.g. alphas[1][0] corresponds to the probability
                        of observing x^1:1, i.e. the first observation,
                        given that y^1 = 0, i.e. the first state is 0.
        '''

        M = len(x)      # Length of sequence.

        alphas = [[0. for _ in range(self.L)] for _ in range(M + 1)]

        # Note that alpha_j(0) is already correct for all j's.
        # Calculate alpha_j(1) for all j's.
        for curr in range(self.L):
            if x[0] in words:
                alphas[1][curr] = self.A_start[curr] * self.O[curr][words[x[0]]]

        # Calculate alphas throughout sequence.
        for t in range(1, M):
            # Iterate over all possible current states.
            for curr in range(self.L):
                prob = 0

                # Iterate over all possible previous states to accumulate
                # the probabilities of all paths from the start state to
                # the current state.
                for prev in range(self.L):
                    if x[t] in words:
                        prob += alphas[t][prev] \
                            * self.A[prev][curr] \
                            * self.O[curr][words[x[t]]]

                # Store the accumulated probability.
                alphas[t + 1][curr] = prob

            if normalize:
                norm = sum(alphas[t + 1])
                for curr in range(self.L):
                    if norm != 0:
                        alphas[t + 1][curr] /= norm

        return alphas


    def backward(self, x, normalize=False):
        '''
        Uses the backward algorithm to calculate the beta probability
        vectors corresponding to a given input sequence.

        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.

            normalize:  Whether to normalize each set of alpha_j(i) vectors
                        at each i. This is useful to avoid underflow in
                        unsupervised learning.

        Returns:
            betas:      Vector of betas.

                        The (i, j)^th element of betas is beta_j(i), i.e.
                        the probability of observing prefix x^(i+1):M and
                        state y^i = j.

                        e.g. betas[M][0] corresponds to the probability
                        of observing x^M+1:M, i.e. no observations,
                        given that y^M = 0, i.e. the last state is 0.
        '''

        M = len(x)      # Length of sequence.
        betas = [[0. for _ in range(self.L)] for _ in range(M + 1)]

        # Initialize initial betas.
        for curr in range(self.L):
            betas[-1][curr] = 1

        # Calculate betas throughout sequence.
        for t in range(-1, -M - 1, -1):
            # Iterate over all possible current states.
            for curr in range(self.L):
                prob = 0

                # Iterate over all possible next states to accumulate
                # the probabilities of all paths from the end state to
                # the current state.
                for nxt in range(self.L):
                    if (x[t] in words):
                        if t == -M:
                            prob += betas[t][nxt] \
                                    * self.A_start[nxt] \
                                    * self.O[nxt][words[x[t]]]

                        else:
                            prob += betas[t][nxt] \
                                    * self.A[curr][nxt] \
                                    * self.O[nxt][words[x[t]]]

                # Store the accumulated probability.
                betas[t - 1][curr] = prob

            if normalize:
                norm = sum(betas[t - 1])
                for curr in range(self.L):
                    if norm != 0:
                        betas[t - 1][curr] /= norm

        return betas


    def supervised_learning(self, X, Y):
        '''
        Trains the HMM using the Maximum Likelihood closed form solutions
        for the transition and observation matrices on a labeled
        datset (X, Y). Note that this method does not return anything, but
        instead updates the attributes of the HMM object.

        Arguments:
            X:          A dataset consisting of input sequences in the form
                        of lists of variable length, consisting of integers
                        ranging from 0 to D - 1. In other words, a list of
                        lists.

            Y:          A dataset consisting of state sequences in the form
                        of lists of variable length, consisting of integers
                        ranging from 0 to L - 1. In other words, a list of
                        lists.

                        Note that the elements in X line up with those in Y.
        '''

        # Calculate each element of A using the M-step formulas.
        for curr in range(self.L):
            for nxt in range(self.L):
                num = 0.
                den = 0.

                for i in range(len(X)):
                    x = X[i]
                    y = Y[i]
                    M = len(x)

                    num += len([1 for i in range(M - 1) \
                                if y[i] == curr and y[i + 1] == nxt])
                    den += len([1 for i in range(M - 1) if y[i] == curr])

                self.A[curr][nxt] = num / den

        # Calculate each element of O using the M-step formulas.
        for curr in range(self.L):
            for xt in range(self.D):
                num = 0.
                den = 0.

                for i in range(len(X)):
                    x = X[i]
                    y = Y[i]
                    M = len(x)

                    num += len([1 for i in range(M) \
                                if y[i] == curr and x[i] == xt])
                    den += len([1 for i in range(M) if y[i] == curr])

                self.O[curr][xt] = num / den


    def unsupervised_learning(self, X, N_iters):
        '''
        Trains the HMM using the Baum-Welch algorithm on an unlabeled
        datset X. Note that this method does not return anything, but
        instead updates the attributes of the HMM object.

        Arguments:
            X:          A dataset consisting of input sequences in the form
                        of lists of length M, consisting of integers ranging
                        from 0 to D - 1. In other words, a list of lists.

            N_iters:    The number of iterations to train on.
        '''

        # Note that a comment starting with 'E' refers to the fact that
        # the code under the comment is part of the E-step.

        # Similarly, a comment starting with 'M' refers to the fact that
        # the code under the comment is part of the M-step.

        for iteration in range(1, N_iters + 1):
            print("\rIteration: " + str(iteration) + ", " + str(100 * iteration / N_iters) + "%", end="")

            # Numerator and denominator for the update terms of A and O.
            A_num = [[0. for i in range(self.L)] for j in range(self.L)]
            O_num = [[0. for i in range(self.D)] for j in range(self.L)]
            A_den = [0. for i in range(self.L)]
            O_den = [0. for i in range(self.L)]

            # For each input sequence:
            for x in X:
                M = len(x)
                # Compute the alpha and beta probability vectors.
                alphas = self.forward(x, normalize=True)
                betas = self.backward(x, normalize=True)

                # E: Update the expected observation probabilities for a
                # given (x, y).
                # The i^th index is P(y^t = i, x).
                for t in range(1, M + 1):
                    P_curr = [0. for _ in range(self.L)]

                    for curr in range(self.L):
                        P_curr[curr] = alphas[t][curr] * betas[t][curr]

                    # Normalize the probabilities.
                    norm = sum(P_curr)
                    for curr in range(len(P_curr)):
                        if norm != 0:
                            P_curr[curr] /= norm

                    for curr in range(self.L):
                        if x[t-1] in words:
                            if t != M:
                                A_den[curr] += P_curr[curr]
                            O_den[curr] += P_curr[curr]
                            O_num[curr][words[x[t - 1]]] += P_curr[curr]

                # E: Update the expectedP(y^j = a, y^j+1 = b, x) for given (x, y)
                for t in range(1, M):
                    P_curr_nxt = [[0. for _ in range(self.L)] for _ in range(self.L)]

                    for curr in range(self.L):
                        for nxt in range(self.L):
                            if x[t] in words:
                                P_curr_nxt[curr][nxt] = alphas[t][curr] \
                                                        * self.A[curr][nxt] \
                                                        * self.O[nxt][words[x[t]]] \
                                                        * betas[t + 1][nxt]

                    # Normalize:
                    norm = 0
                    for lst in P_curr_nxt:
                        norm += sum(lst)
                    for curr in range(self.L):
                        for nxt in range(self.L):
                            if norm != 0:
                                P_curr_nxt[curr][nxt] /= norm

                    # Update A_num
                    for curr in range(self.L):
                        for nxt in range(self.L):
                            A_num[curr][nxt] += P_curr_nxt[curr][nxt]

            for curr in range(self.L):
                for nxt in range(self.L):
                    if A_den[curr] != 0:

                        self.A[curr][nxt] = A_num[curr][nxt] / A_den[curr]

            for curr in range(self.L):
                for xt in range(self.D):
                    if O_den[curr] != 0:
                        self.O[curr][xt] = O_num[curr][xt] / O_den[curr]

    def generate_emission(self, n_syllables):
        '''
        Generates an emission of length M, assuming that the starting state
        is chosen uniformly at random.

        Arguments:
            n_syllables: num of syllables of the emission to generate.

        Returns:
            emission:   The randomly generated emission as a list.

            states:     The randomly generated states as a list.
        '''

        emission = []
        state = random.choice(range(self.L))
        states = []

        first_words_sylls = {0} # total number of sylls without last_word
        last_word_sylls = {0} # potential number of syllables of last_word
        while max(first_words_sylls) + max(last_word_sylls) < n_syllables:
            # Append state.
            states.append(state)

            # Sample next observation.
            rand_var = random.uniform(0, 1)
            next_obs = 0

            while rand_var > 0:
                rand_var -= self.O[state][next_obs]
                next_obs += 1

            next_obs -= 1 # the id of the next word
            emission.append(next_obs)
            next_word = word_list[next_obs]

            # update the total first_words_sylls
            new_counts = set()
            for n in last_word_sylls:
                for m in first_words_sylls:
                    new_counts.add(m + n)
            first_words_sylls = new_counts

            # update last_word_sylls
            last_word_sylls = set()
            for item in word_sylls[next_word]:
                if item[0] == "E":
                    n = int(item[1])
                else:
                    n = int(item)
                last_word_sylls.add(n)

            # Sample next state.
            rand_var = random.uniform(0, 1)
            next_state = 0

            while rand_var > 0:
                rand_var -= self.A[state][next_state]
                next_state += 1

            next_state -= 1
            state = next_state

        # If the number of syllables does not match exactly, change last word
        total_sylls = set()
        for m in first_words_sylls:
            for n in last_word_sylls:
                total_sylls.add(m + n)
        while not n_syllables in total_sylls:
            # revert to previous state
            emission = emission[: -1]
            state = states[len(states) - 2]

            # Sample next state.
            rand_var = random.uniform(0, 1)
            next_state = 0
            while rand_var > 0:
                rand_var -= self.A[state][next_state]
                next_state += 1
            next_state -= 1
            state = next_state

            # Append state.
            states.append(state)

            # Sample next observation.
            rand_var = random.uniform(0, 1)
            next_obs = 0

            while rand_var > 0:
                rand_var -= self.O[state][next_obs]
                next_obs += 1

            next_obs -= 1 # the id of the next word
            emission.append(next_obs)
            next_word = word_list[next_obs]

            # update total_sylls
            total_sylls = set()
            for item in word_sylls[next_word]:
                if item[0] == "E":
                    n = int(item[1])
                else:
                    n = int(item)
                for m in first_words_sylls:
                    total_sylls.add(m + n)

        return emission, states


    def probability_alphas(self, x):
        '''
        Finds the maximum probability of a given input sequence using
        the forward algorithm.

        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.

        Returns:
            prob:       Total probability that x can occur.
        '''

        # Calculate alpha vectors.
        alphas = self.forward(x)

        # alpha_j(M) gives the probability that the output sequence ends
        # in j. Summing this value over all possible states j gives the
        # total probability of x paired with any output sequence, i.e. the
        # probability of x.
        prob = sum(alphas[-1])
        return prob


    def probability_betas(self, x):
        '''
        Finds the maximum probability of a given input sequence using
        the backward algorithm.

        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.

        Returns:
            prob:       Total probability that x can occur.
        '''

        betas = self.backward(x)

        # beta_j(0) gives the probability of the output sequence. Summing
        # this over all states and then normalizing gives the total
        # probability of x paired with any output sequence, i.e. the
        # probability of x.
        prob = sum([betas[1][k] * self.A_start[k] * self.O[k][x[0]] \
            for k in range(self.L)])

        return prob


def supervised_HMM(X, Y):
    '''
    Helper function to train a supervised HMM. The function determines the
    number of unique states and observations in the given data, initializes
    the transition and observation matrices, creates the HMM, and then runs
    the training function for supervised learning.

    Arguments:
        X:          A dataset consisting of input sequences in the form
                    of lists of variable length, consisting of integers
                    ranging from 0 to D - 1. In other words, a list of lists.

        Y:          A dataset consisting of state sequences in the form
                    of lists of variable length, consisting of integers
                    ranging from 0 to L - 1. In other words, a list of lists.
                    Note that the elements in X line up with those in Y.
    '''

    # Make a set of observations.
    observations = set()
    for x in X:
        observations |= set(x)

    # Make a set of states.
    states = set()
    for y in Y:
        states |= set(y)

    # Compute L and D.
    L = len(states)
    D = len(observations)

    # Randomly initialize and normalize matrices A and O.
    A = [[random.random() for i in range(L)] for j in range(L)]

    for i in range(len(A)):
        norm = sum(A[i])
        for j in range(len(A[i])):
            A[i][j] /= norm

    O = [[random.random() for i in range(D)] for j in range(L)]

    for i in range(len(O)):
        norm = sum(O[i])
        for j in range(len(O[i])):
            O[i][j] /= norm

    # Train an HMM with labeled data.
    HMM = HiddenMarkovModel(A, O)
    HMM.supervised_learning(X, Y)

    return HMM


def unsupervised_HMM(X, n_states, N_iters):
    '''
    Helper function to train an unsupervised HMM. The function determines the
    number of unique observations in the given data, initializes
    the transition and observation matrices, creates the HMM, and then runs
    the training function for unsupervised learing.

    Arguments:
        X:          A dataset consisting of input sequences in the form
                    of lists of variable length, consisting of integers
                    ranging from 0 to D - 1. In other words, a list of lists.

        n_states:   Number of hidden states to use in training.

        N_iters:    The number of iterations to train on.
    '''

    # Make a set of observations.
    observations = set()
    for x in X:
        observations |= set(x)

    # Compute L and D.
    L = n_states
    D = len(observations)

    # Randomly initialize and normalize matrices A and O.
    A = [[random.random() for i in range(L)] for j in range(L)]

    for i in range(len(A)):
        norm = sum(A[i])
        for j in range(len(A[i])):
            A[i][j] /= norm

    # Randomly initialize and normalize matrix O.
    O = [[random.random() for i in range(len(words))] for j in range(L)]

    for i in range(len(O)):
        norm = sum(O[i])
        for j in range(len(O[i])):
            O[i][j] /= norm

    # Train an HMM with unlabeled data.
    HMM = HiddenMarkovModel(A, O)
    HMM.unsupervised_learning(X, N_iters)

    return HMM


In [2]:
'''
Preprocessing the poems.
Example usage:

import preprocessing as pp
X = pp.load_poems("data/shakespeare.txt")

'''

SYMBOL_LIST = {'?', '.',  ')', ',', '!', ';', '(', ':'}

word_list = []
word_sylls = {}
with open("data/Syllable_dictionary.txt") as f:
    while True:
        line = f.readline()
        if line == "":
            break
        line = line.strip()
        lst = line.split(" ", 1)
        if len(lst) != 2:
            print(lst)
        word_list.append(lst[0])
        word_sylls[lst[0]] = lst[1].split(" ")

def load_poems(filename, onepoem=False):
    '''
    filename should be directory/filename.
    if onepoem, then only return data for the 1st poem.
    Returns the matrix X, where each row is a line of a poem, and each col is
            a word of the line.
    '''
    with open(filename) as f:
        # The matrix consisting of all lines
        X = []
        while True:
            line = f.readline()
            if line == "":
                break
            # Remove whitespaces from the front and back
            line = line.strip()

            if onepoem and line == "2":
                break

            # If it is not an actual poem line
            if len(line) < 5:
                continue
            # Remove punctuations
            for symbol in SYMBOL_LIST:
                line = line.replace(symbol, "")
            # Convert to lowercase
            line = line.lower()
            # Split into an array of words
            x = line.split(" ")
            # Check if it is in the Syllable_dictionary
            for i in range(len(x)):
                if not x[i] in word_list:
                    x[i] = x[i].replace("\'", "")
            # Add to the matrix of lines
            X.append(x)
    return X

def list_of_words():
    '''
    returns a list that maps id to word
    '''
    return word_list

def word_id():
    '''
    returns a dictionary that maps word to id
    '''
    dictionary = {}
    for i in range(len(word_list)):
        word = word_list[i]
        dictionary[word] = i
    return dictionary

def word_syllables():
    '''
    returns a dictionary that maps word to a list of potential number of
            syllables. Each element in the list is a string.
    '''
    return word_sylls

In [3]:
word_list = list_of_words()   # id   --> word
word_id = word_id()           # word --> id
word_sylls = word_syllables() # word --> n_syllables


poem_lines = load_poems("shakespeare.txt")

# Train HMM
hmm = unsupervised_HMM(poem_lines, 10, 50)


def generate_line(model, n_syllables):

    emissions, states = model.generate_emission(n_syllables)
    line = ""
    for i in emissions:
        line += word_list[i]
        line += " "
    return line

# Write a sonnet
print()
for i in range(14):
    if i % 2 == 0:
        # even lines add comma (because it starts with 0)
        line = generate_line(hmm, 10).capitalize().strip() + ","
    else:
        # odd lines add period
        line = generate_line(hmm, 10).capitalize().strip() + "."
    print(line)


Iteration: 50, 100.0%
Still and mars my naked my flower veil and,
Keen yet it defy thus for heaven's thine.
More in thee to-morrow weep to that waste,
O for thy wilfulness when of early.
Spot upon thy disdain in mourning stelled,
Great name green and this thou statute the glass.
With crooked even of much thoughts doth ever,
In with ill onwards is and wherefore i.
Controlling be nor than mayst most at love's,
Sufferance up songs doth her take poet's fair.
Deeds thought thrust all shorn of two not do choose,
Speaking worse mind our of pilgrimage.
Thy self born thou in thee all which my thrall,
Or divide victor the worst seen thy turns.


In [14]:
print(generate_line(hmm,17))

fear the past says worth to my favourites faults and in eternity 


In [ ]:
fear the past says worth to my favourites faults and in eternity 

In [16]:
print(generate_line(hmm,17))

thence rocks summer's life shall air take bootless shapes shame weigh night thousand spirit 


In [32]:
t = hmm.A
for i in range(len(hmm.A)):
    for j in range(len(hmm.A[0])):
        if t[i][j] < 10**(-4): 
            t[i][j] = 0

In [40]:
import numpy as np
ao = np.dot(hmm.A,hmm.O)

In [61]:
for i in range(10):
    print(ao[:,i], word_list[i])

[  4.28209344e-04   6.87007824e-06   1.39372920e-04   8.90238421e-05
   1.27471445e-35   2.86136349e-06   7.98672251e-06   1.02346453e-03
   6.07338998e-04   1.41748277e-03] 'gainst
[  9.61026512e-05   8.65775178e-05   1.38934577e-04   3.06427859e-04
   1.76676365e-85   5.44524605e-06   4.79717450e-25   1.30516338e-05
   1.90940753e-64   1.17962802e-36] 'greeing
[  2.71105126e-04   6.02716288e-25   8.13056920e-26   7.48122926e-19
   6.55222364e-88   3.32704444e-07   2.74575921e-15   2.21129524e-05
   2.74656499e-04   1.99050617e-26] 'scaped
[  1.08738756e-03   3.30808153e-04   1.05006540e-03   1.18747687e-03
   1.37567669e-04   3.15300051e-04   1.36943260e-22   3.66013262e-04
   1.24736778e-03   2.35652426e-03] 'tis
[  1.35560144e-004   2.12683608e-016   7.24948271e-016   3.61104147e-006
   4.36847119e-106   1.70250143e-007   4.54573439e-017   6.02071048e-004
   2.52924810e-004   7.93011539e-005] 'twixt
[  1.71200271e-02   6.31101522e-05   1.55076804e-02   1.07670607e-02
   4.61000925e

In [86]:
word_id['we']

3025

In [87]:
#pronouns
for i in [1365, 2391, 1269,2738,3025]:
    print( word_list[i],np.argmax(ao[:,i]),ao[:,i])

i 8 [ 0.04432326  0.00423788  0.02677486  0.01212678  0.01019067  0.01749868
  0.00042165  0.02100358  0.06885     0.00767858]
she 4 [  2.41390025e-03   3.74993977e-12   2.94042269e-03   2.19998842e-03
   4.88334357e-03   3.84230668e-04   1.31570454e-03   3.79847590e-03
   1.55327432e-11   2.24408578e-12]
he 9 [  1.94834956e-03   1.04037337e-03   3.54749913e-03   3.87216666e-03
   1.93170158e-03   9.46541457e-04   2.84962620e-06   2.35724919e-03
   2.64537765e-03   1.09819553e-02]
they 8 [  5.13660524e-03   8.60128902e-04   3.25910616e-03   1.25933057e-03
   9.60415465e-04   2.53851845e-03   8.22716024e-08   4.54493155e-03
   8.60477364e-03   5.59613610e-03]
we 8 [  2.38859042e-03   5.24924364e-04   6.91895596e-05   1.22656851e-04
   7.41785851e-24   6.25585151e-04   1.61751407e-30   1.76658741e-03
   2.71840796e-03   2.10993179e-04]


In [74]:
word_id

{"'gainst": 0,
 "'greeing": 1,
 "'scaped": 2,
 "'tis": 3,
 "'twixt": 4,
 'a': 5,
 'a-doting': 6,
 'abhor': 7,
 'abide': 8,
 'able': 9,
 'about': 10,
 'above': 11,
 'absence': 12,
 'absent': 13,
 'abundance': 14,
 'abundant': 15,
 'abuse': 16,
 'abused': 17,
 'abuses': 18,
 'abysm': 19,
 'accents': 20,
 'acceptable': 21,
 'acceptance': 22,
 'accessary': 23,
 'accident': 24,
 'accidents': 25,
 'account': 26,
 'accumulate': 27,
 'accuse': 28,
 'accusing': 29,
 'achieve': 30,
 'acknowledge': 31,
 'acquaintance': 32,
 'acquainted': 33,
 'act': 34,
 'action': 35,
 'active': 36,
 'actor': 37,
 'add': 38,
 'added': 39,
 "adder's": 40,
 'addeth': 41,
 'adding': 42,
 'addition': 43,
 'adieu': 44,
 'adjunct': 45,
 'admire': 46,
 'admired': 47,
 'admiring': 48,
 'admit': 49,
 'admitted': 50,
 'adonis': 51,
 'adore': 52,
 'adulterate': 53,
 'advance': 54,
 'advantage': 55,
 'adverse': 56,
 'advised': 57,
 'advocate': 58,
 'afar': 59,
 'affable': 60,
 'affairs': 61,
 'affections': 62,
 'afford': 63,

In [78]:
verb = [7,8,17,27,28,31,38,39,46,47,52,54,90,91,97]

In [81]:
indices = []
for i in range(len(verb)):
    print( word_list[verb[i]],ao[:,verb[i]])
    indices.append(np.argmax(ao[:,verb[i]]))

abhor [  6.46432887e-06   3.81356499e-04   5.02660772e-05   8.21721488e-05
   2.09800770e-04   4.52357186e-04   3.82777285e-07   1.19958145e-10
   5.03925384e-11   1.57998988e-11]
abide [  1.18886236e-04   1.44983645e-05   2.69261162e-04   1.48273805e-08
   1.79627334e-04   2.27662029e-04   1.34502637e-07   5.45735844e-05
   4.62491785e-04   1.06251400e-28]
abused [  9.61026512e-05   8.65775178e-05   1.38934577e-04   3.06427859e-04
   1.10726721e-45   5.44524605e-06   4.26330357e-22   1.30516338e-05
   1.56503932e-51   1.37384630e-34]
accumulate [  9.44428506e-039   1.15174550e-039   2.13900218e-038   1.11356325e-007
   5.53658759e-004   1.80853997e-038   1.01013927e-006   4.33530833e-039
   3.67402015e-038   2.24844338e-176]
accuse [  1.35151321e-04   4.45208030e-07   8.26832787e-06   3.50310628e-06
   3.25211248e-06   7.15607558e-06   1.09208491e-06   5.85701446e-04
   2.59549551e-04   7.69242823e-05]
acknowledge [  6.77800718e-05   1.65209886e-46   8.92467550e-42   1.80552074e-06
  

In [82]:
indices

[5, 8, 3, 4, 7, 7, 4, 3, 8, 7, 7, 4, 1, 1, 6]

NameError: name 'sort' is not defined